In [4]:
from pathlib import Path
import datetime
from zoneinfo import ZoneInfo
import shutil
from neuroconv.utils import load_dict_from_file, dict_deep_update
import h5py
import numpy as np
from pynwb import NWBHDF5IO, NWBFile
import os
import pynwb
import scipy.io as sio

In [ ]:
file = h5py.File('E:\\odor-pixels\\M541-2024-08-31\\imec0_clean_lfp.mat', 'r') 

In [ ]:
file['imec0'].keys()

In [ ]:
# Code to get channel ids in array
q = np.asarray(file['imec0']['channel_ids'][:], dtype='uint32')
q = q.T.view('U1')
channel_ids = np.asarray([''.join(x).strip() for x in q])
channel_ids

In [ ]:
# Code to get shank_ids in array
shank_id = file['imec0']['shank_ids'][:].flatten()
shank_id


In [ ]:
# Code to get depths ids in array TODO: Needs subtraction from ExpKeys.recordingDepth
depths = file['imec0']['depths'][:].flatten()
depths

In [ ]:
# Code to get lfp_fs
lfp_fs = file['imec0']['lfp_fs'][:].flatten()
lfp_fs[0]

In [ ]:
# Code to get lfp_tvec in array
lfp_tvec = file['imec0']['lfp_tvec'][:].flatten()
lfp_tvec

In [ ]:
# Code to get lfp_traces in array
lfp_data= file['imec0']['lfp_traces'][:]
lfp_data = np.transpose(lfp_data)
lfp_data

In [ ]:
from uuid import uuid4
from datetime import datetime
from dateutil.tz import tzlocal

In [ ]:
nwbfile = NWBFile(
    session_description="my first synthetic recording",
    identifier='-'.join([expkeys['subject'], expkeys['date']]),
    session_start_time=datetime.now(tzlocal()), # Get from .meta file
    experimenter=[
        ""Mohapatra, Manish","
    ],
    lab="vandermeerlab",
    institution="Dartmouth College",
    experiment_description="Head-fixed mouse presented with odor sequences",
    keywords=["ecephys", "exploration", "wanderlust"],
)
device = nwbfile.create_device(
    name="imec0", description="NPX2.0", manufacturer="IMEC"
)
nwbfile.add_electrode_column(name="channel_id", description="Identifier for the channel on the probe")

unique_shanks = np.unique(shank_id).tolist()
electrode_counter = 0
for iShank in unique_shanks:
    electrode_group = nwbfile.create_electrode_group(
        name="shank{}".format(iShank),
        description="electrode group for shank {}".format(iShank),
        device=device,
        location="brain area", # Need to figure this out, should this be the same as depth
    )
    # add electrodes to the electrode table
    for ielec,elec in enumerate(np.where(shank_id == iShank)[0]):
        # print
        nwbfile.add_electrode(
            group=electrode_group,
            channel_id = channel_ids[elec],
            location="brain area",  # Need to figure this out, should this be the same as depth
        )
        electrode_counter += 1

# nwbfile.electrodes.to_dataframe() # Distplay the electrode table

In [ ]:
lfp_table = nwbfile.create_electrode_table_region(
    region=list(range(electrode_counter)),  # reference row indices 0 to N-1
    description="LFP electrodes",
)

In [ ]:
from pynwb.ecephys import ElectricalSeries, LFP
lfp_es = ElectricalSeries(name='LFP', data=lfp_data, electrodes=lfp_table, rate=lfp_fs[0], starting_time=lfp_tvec[0])
ecephys_module = nwbfile.create_processing_module(
    name="ecephys", description="processed extracellular electrophysiology data"
)
ecephys_module.add(lfp_es)
# TODO: Add conversion factor from .meta file , but we already get 'scaled data' from spikeinterface'

In [ ]:
with NWBHDF5IO("E:\\odor-pixels\\M541-2024-08-31\\test.nwb", "w") as io:
    io.write(nwbfile)

In [ ]:
with NWBHDF5IO("E:\\odor-pixels\\M541-2024-08-31\\test.nwb", "r") as io:
    nwbfile = io.read()

In [ ]:
io = NWBHDF5IO("E:\\odor-pixels\\M541-2024-08-31\\test.nwb", "r")
nwb_file = io.read()
nwb_file

In [ ]:
pynwb.validate(nwb_file)

In [2]:
os.path.exists('E:\\odor-pixels\\M541-2024-08-31\\clean_units_imec0.mat')

True

In [5]:
# Spiking data is in older_matlab style
spike_data = sio.loadmat('E:\\odor-pixels\\M541-2024-08-31\\clean_units_imec0.mat')
spike_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'depths', 'unit_ids', 'channel_ids', 'spike_train', 'shank_ids', 'mean_waveforms'])

In [17]:
# Code to extract average waveform
all_wv = spike_data['mean_waveforms'].squeeze()
big_idx = [np.argmax(abs(np.max(all_wv[x], axis=1) - np.min(all_wv[x], axis=1))) for x in range(all_wv.shape[0])]
big_wv = np.asarray([all_wv[x][y][:] for x,y in enumerate(big_idx)])
big_wv.shape

(116, 90)

In [31]:
spike_trains[0][0].squeeze().shape

(4742,)

In [21]:
# Code to get spike_trains
spike_trains = spike_data['spike_train'].T

In [ ]:
# Code to get depths #TODO: Needs subtraction from ExpKeys.recordingDepth 
spike_data['depths'].squeeze()

In [10]:
# Code to get shank_ids #TODO: Need to add device + shank
spike_data['shank_ids'].squeeze()

array([2, 2, 2, 2, 2, 2, 3, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3,
       3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2,
       3, 3, 3, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 2, 0, 1, 1, 3, 0,
       0, 0, 0, 0, 3, 0, 3, 1, 0, 0, 0, 0, 1, 1, 1, 3, 0, 1, 0, 0, 0, 2,
       1, 0, 0, 0, 3, 1])

In [9]:
# Code to get channel_ids
spike_data['channel_ids'].shape

(116,)

In [15]:
spike_data['unit_ids']

array(['imec0_0  ', 'imec0_3  ', 'imec0_4  ', 'imec0_5  ', 'imec0_6  ',
       'imec0_8  ', 'imec0_23 ', 'imec0_40 ', 'imec0_53 ', 'imec0_54 ',
       'imec0_62 ', 'imec0_63 ', 'imec0_64 ', 'imec0_69 ', 'imec0_72 ',
       'imec0_73 ', 'imec0_74 ', 'imec0_76 ', 'imec0_77 ', 'imec0_79 ',
       'imec0_94 ', 'imec0_97 ', 'imec0_99 ', 'imec0_109', 'imec0_110',
       'imec0_114', 'imec0_115', 'imec0_116', 'imec0_117', 'imec0_118',
       'imec0_122', 'imec0_139', 'imec0_140', 'imec0_142', 'imec0_146',
       'imec0_149', 'imec0_153', 'imec0_155', 'imec0_160', 'imec0_161',
       'imec0_165', 'imec0_170', 'imec0_175', 'imec0_176', 'imec0_177',
       'imec0_184', 'imec0_188', 'imec0_189', 'imec0_192', 'imec0_197',
       'imec0_200', 'imec0_201', 'imec0_210', 'imec0_211', 'imec0_212',
       'imec0_213', 'imec0_214', 'imec0_216', 'imec0_220', 'imec0_231',
       'imec0_234', 'imec0_237', 'imec0_243', 'imec0_246', 'imec0_248',
       'imec0_257', 'imec0_261', 'imec0_263', 'imec0_264', 'imec